In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
#import yfinance as yf
import os

# Parameter
#symbol = 'TSLA'
pred_days = 100
look_back = 60


# 1. Daten abrufen
folderPath_His = "/Users/Shared/ai_work/Trainingdata/ml_data/yh_his/D1/"
folderPath_models = "/Users/Shared/ai_work/Trainingdata/models/rnn/"
folderpath_results = "/Users/Shared/ai_work/Trainingdata/ml_results/rnn/"

#"/Users/workplacelivetv/Library/CloudStorage/GoogleDrive-robo01.rpa@gmail.com/My Drive/ml_data/yh_his/D1/"
#filepath = "/Users/workplacelivetv/Library/CloudStorage/GoogleDrive-robo01.rpa@gmail.com/My Drive/ml_data/yh_his/D1/QBTS.csv"
symbol = '#PLTR'
df = pd.read_csv (folderPath_His + symbol + ".csv")
#df = pd.read_csv (filepath)
# Daten laden
#df = yf.download("TSLA", start="2018-01-01", end=None)

lastBars = 0
n_tail = 200
n_pred_days = 60
n_time_steps = 60 # Neuronal Netzwerk
sequence_length = 60
n_epochs = 200 
n_batch_size = 16
df_his = df.tail(lastBars+1)
df_his["date"] = pd.to_datetime(df_his["date"]) 


df = df[['date','open', 'high', 'low', 'close','volume']]
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
data = df.copy()

#data = df.copy()
if n_tail > 0: data = data.tail(n_tail)
if lastBars > 0: df = df[: len(data)-lastBars]
#data = df[['close']].dropna()

# 2. Technische Indikatoren berechnen
data['SMA_20'] = data['close'].rolling(window=20).mean()
change = data['close'].diff()
gain = change.where(change > 0, 0).rolling(window=14).mean()
loss = (-change.where(change < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))
ema12 = data['close'].ewm(span=12, adjust=False).mean()
ema26 = data['close'].ewm(span=26, adjust=False).mean()
data['MACD'] = ema12 - ema26

data.fillna(method='bfill', inplace=True)

# 3. Features auswählen
features = ['open', 'high', 'low', 'close', 'volume', 'SMA_20', 'RSI', 'MACD']
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[features])

# 4. Sequenzen für LSTM vorbereiten
X, y = [], []
for i in range(look_back, len(scaled_data) - pred_days):
    X.append(scaled_data[i - look_back:i])
    y.append(scaled_data[i:i + pred_days, 3])
X, y = np.array(X), np.array(y)

# 5. Modell erstellen
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(pred_days)
])
model.compile(optimizer='adam', loss='mse')

# 6. Training
model.fit(X, y, epochs=10, batch_size=32, verbose=1)

# 7. Vorhersage für die nächsten 100 Handelstage
last_seq = scaled_data[-look_back:]
last_seq = np.expand_dims(last_seq, axis=0)
pred_scaled = model.predict(last_seq)[0]

# Rückskalieren
scaler_close = MinMaxScaler()
scaler_close.fit(data[['close']])
pred_prices = scaler_close.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()

# 8. Business-Day-Achse
future_dates = pd.date_range(start=data.index[-1], periods=pred_days+1, freq='B')[1:]



Epoch 1/10


/tmp/ipykernel_328167/3627197082.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_328167/3627197082.py:61: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/home/aiubuntu01/anaconda3/envs/tf-gpu/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.4320
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.3759
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.3075
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.2251
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.1593
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.1265
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0937
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0820
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0691
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0586
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


In [7]:

# 9. Plot erstellen
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index[-200:], y=data['close'][-200:], name='Historische Kurse'))
fig.add_trace(go.Scatter(x=future_dates, y=pred_prices, name='Prognose (100 Tage)'))
fig.update_layout(title='Tesla Aktienkurs Prognose mit LSTM', xaxis_title='Datum', yaxis_title='Preis (USD)')
#fig.write_image('tesla_forecast.png')
#fig.show()
pred_prices
print("Vorhersage abgeschlossen. Plot gespeichert als tesla_forecast.png")

Vorhersage abgeschlossen. Plot gespeichert als tesla_forecast.png


In [8]:
pred_prices

array([118.177246, 116.77951 , 130.88943 , 136.55763 , 114.50538 ,
       140.48602 , 139.09131 , 127.72241 , 113.08691 , 122.33224 ,
       128.2142  , 126.294716, 127.86634 , 133.94418 , 129.08917 ,
       140.31357 , 152.50378 , 146.39069 , 135.5076  , 120.554504,
       141.56844 , 127.832726, 145.49757 , 133.86577 , 131.79922 ,
       149.50912 , 126.376335, 130.9237  , 155.69914 , 149.71056 ,
       144.34622 , 153.13986 , 156.49513 , 145.29294 , 151.9641  ,
       147.4032  , 156.47327 , 152.4198  , 169.2107  , 175.04431 ,
       154.18175 , 148.6263  , 160.19121 , 174.88757 , 149.77174 ,
       169.3794  , 160.68625 , 161.1202  , 160.00096 , 164.69562 ,
       177.3905  , 155.13544 , 152.19456 , 165.19275 , 178.02548 ,
       146.78297 , 146.79272 , 147.93788 , 169.95172 , 147.73685 ,
       166.76526 , 171.79997 , 169.98303 , 128.02754 , 175.40968 ,
       155.05798 , 163.95206 , 170.11963 , 156.9689  , 158.37032 ,
       157.00156 , 158.7986  , 151.48518 , 171.7808  , 161.957